In [2]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

train = pd.read_csv("./data/sales_train.csv")
test = pd.read_csv("./data/test.csv", index_col='ID')
cat = pd.read_csv("./data/items.csv", index_col='item_id')

In [3]:
train_clip = train.copy()
train_clip["item_cnt_day"] = train_clip["item_cnt_day"].clip(0, 20)

group = train_clip.groupby(["shop_id", "item_id", "date_block_num"])
train1 = group.agg({'item_cnt_day':'sum', 'item_price':'median'}).reset_index()
train1 = train1.rename(columns={"item_cnt_day": "item_cnt_month"})
train1["item_cnt_month"] = train1["item_cnt_month"].clip(0, 20)

In [4]:
train_sub = train1[train1["date_block_num"] == 33][["shop_id", "item_id", "item_cnt_month"]]
train_sub

,shop_id,item_id,item_cnt_month
8113,2,31,1.0
8205,2,486,3.0
8254,2,787,1.0
8284,2,794,1.0
8367,2,968,1.0
...,...,...,...
1608998,59,22087,6.0
1609030,59,22088,2.0
1609047,59,22091,1.0
1609073,59,22100,1.0


In [ ]:
display(test)
test1 = test.merge(train_sub, on=["shop_id", "item_id"], how='left')
test1 = test1.fillna(0).clip(0, 20)
test1.index = test.index
display(test1)
test1["item_cnt_month"].to_csv("benchmark.csv")

In [5]:
train1[(train1["shop_id"] == 59) & (train1["item_id"] == 22167)].iloc[-1]

shop_id              59.0
item_id           22167.0
date_block_num       17.0
item_cnt_month        1.0
item_price          299.0
Name: 1609123, dtype: float64

In [6]:
def find_latest_entry(row):
    row = pd.DataFrame(row).T
    shop = int(row["shop_id"])
    item = int(row["item_id"])
    
    try:
        
        train_rows = train1[(train1["shop_id"] == shop) & (train1["item_id"] == item)]
        last_row = train_rows.iloc[-1]
        last_row = pd.DataFrame(last_row).T
        last_row["item_cnt_month"] = last_row["item_cnt_month"].clip(0, 20)
        
        last_sale = int(last_row["item_cnt_month"])
        return last_sale
    
    except:
        return 0

test2 = test.copy()
test2["item_cnt_month"] = test2.apply(find_latest_entry, axis=1)
test2

,shop_id,item_id,item_cnt_month
ID,,,
0,5,5037,1
1,5,5320,0
2,5,5233,1
3,5,5232,1
4,5,5268,0
...,...,...,...
214195,45,18454,1
214196,45,16188,0
214197,45,15757,1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:   33.0s remaining:   16.5s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   46.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   46.6s finished


8.382080558353756

[CV] START .....................................................................
[20:06:17] WARNING: ../src/learner.cc:627: 
Parameters: { "random_sate" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[CV] END ............................... score: (test=-8.754) total time=  17.0s
[CV] START .....................................................................
[20:06:31] WARNING: ../src/learner.cc:627: 
Parameters: { "random_sate" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[CV] END ........................